# LoRA Fine-Tuning Qwen-Chat(v2.5) Large Language Model (Multiple GPUs)

Tongyi Qianwen is a large language model developed by Alibaba Cloud based on the Transformer architecture, trained on an extensive set of pre-training data. The pre-training data is diverse and covers a wide range, including a large amount of internet text, specialized books, code, etc. In addition, an AI assistant called Qwen-Chat has been created based on the pre-trained model using alignment mechanism.

This notebook uses Qwen-1.8B-Chat as an example to introduce how to LoRA fine-tune the Qianwen model using Deepspeed.


## 1.Preparation

### 1.1 Download Qwen2.5-1.5B-Chat(Instruct)

First, download the model files. You can choose to download directly from ModelScope or huggingface.

In [ ]:
!huggingface-cli download "Qwen/Qwen2.5-1.5B-Instruct" --local-dir qwen2.5-1.5b-ins

### 1.2 Download the dataset
There are massive datasets generated by R1, e.g.
- bespokelabs/Bespoke-Stratos-17k
- bespokelabs/Bespoke-Stratos-35k
- NovaSky-AI/Sky-T1_data_17k
- open-thoughts/OpenThoughts-114k
- (You can merge these datasets to achieve better results. And less overfitting)

In [ ]:
!huggingface-cli download "bespokelabs/Bespoke-Stratos-17k" --local-dir dataset_stratos_17k --repo-type dataset

In [ ]:
model_name_or_path = "qwen2.5-1.5b-ins" # provide here to merge the model after training

## 2. Process the dataset(Not necessary! Just to remove the **special token** <|begin_of_solution|> and <|end of solution|>)
<br>
Remove the <|begin_of_solution|> and <|end_of_solution|>, but keep the <|begin_of_thought|> and <|end_of_thought|>
<br>
If you have previously processed or merged the datasets, you will need to modify a line of code in `simplify_dataset.py`

In [ ]:
!python simplify_dataset.py

## 3. Add special token to the model

### 3.1 With huggingface official method

you can add the special token `<|begin_of_thought|>` and `<|end_of_thought|>` with python
Ref: https://huggingface.co/docs/transformers/en/main_classes/tokenizer

In [ ]:
from transformers import AutoTokenizer
thought_tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    extra_special_tokens={"thought_begin": "<|begin_of_thought|>", "thought_end": "<|end_of_thought|>", "begin_solution": "<|begin_of_solution|>", "end_solution": '<|end_of_solution|>'}
)
print(thought_tokenizer.thought_begin, thought_tokenizer.thought_begin_id) # should output 151665 with <|begin_of_thought|>

Save the tokenizer(special token added)

In [ ]:
thought_tokenizer.save_pretrained("output_qwen_merged") # your final model

### 3.2 Or, Alternatively you can add the special token by manually editing the tokenizer_config.json
Not ideal, but feasible

## 4. Launch the training process
### ! ! ! Remember to modify your config here, i.e. **model_name_or_path, data_path, nproc_per_node, per_device_train_batch_size, model_max_length...**

In [ ]:
!torchrun --nproc_per_node 2 --nnodes 1 --node_rank 0 --master_addr localhost --master_port 6601 finetune.py \
    --model_name_or_path "qwen2.5-1.5b-ins" \
    --data_path "optimized_dataset" \
    --bf16 True \
    --output_dir "output_qwen" \
    --num_train_epochs 5 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 16 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 500 \
    --save_total_limit 3 \
    --learning_rate 1e-5 \
    --weight_decay 0.1 \
    --adam_beta2 0.95 \
    --warmup_ratio 0.01 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --report_to "none" \
    --model_max_length 8192 \
    --gradient_checkpointing True \
    --lazy_preprocess True \
    --deepspeed "ds_config_zero2.json" \
    --use_lora

## 5. Merge Weights(Merge the Qwen Lora Adapters to the Qwen model) and save tokenizer

The training of both LoRA and Q-LoRA only saves the adapter parameters. You can load the fine-tuned model and merge weights as shown below:

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch

model = AutoModelForCausalLM.from_pretrained("qwen2-5-14b", torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)
model = PeftModel.from_pretrained(model, "output_qwen/")
merged_model = model.merge_and_unload()
merged_model.save_pretrained("output_qwen_merged", max_shard_size="4096MB", safe_serialization=True)

The tokenizer files are not saved in the new directory in this step. You can copy the tokenizer files or use the following code:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    trust_remote_code=True
)

tokenizer.save_pretrained("output_qwen_merged") # your final model

('output_qwen_merged/tokenizer_config.json',
 'output_qwen_merged/special_tokens_map.json',
 'output_qwen_merged/vocab.json',
 'output_qwen_merged/merges.txt',
 'output_qwen_merged/added_tokens.json',
 'output_qwen_merged/tokenizer.json')